In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install datatable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 7.1 MB/s eta 0:00:00


In [3]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("/content/drive/MyDrive/millennium_project")


import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [6]:
ticker_name = "GOOG"

In [7]:
N_FOLD = 10
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = ticker_name+'-1dCNN'
DATA_NAME = ticker_name
mkdir(f'/content/drive/MyDrive/millennium_project/models/{SOL_NAME}/')

# Functions

In [8]:
def rmse_keras(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

In [10]:
def create_1dcnn(num_columns, num_labels, learning_rate):
    # input
    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    # 1dcnn
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Reshape((16, 16))(x)
    x = tf.keras.layers.Conv1D(filters=12,
                      kernel_size=2,
                      strides=1,
                      activation='swish')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)

    # ffn
    for i in range(3):
        x = tf.keras.layers.Dense(64 // (2 ** i), activation='swish')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.GaussianNoise(0.01)(x)
        x = tf.keras.layers.Dropout(0.20)(x)
    x = tf.keras.layers.Dense(num_labels)(x)

    model = tf.keras.models.Model(inputs=inp, outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=rmse_keras,
    )
    return model

# Loading data

In [11]:
# train
df_train = dt.fread(f'./drive/MyDrive/millennium_project/dataset/train_{DATA_NAME}_NN.csv').to_pandas()
fea_cols = fea_cols_GOOG
df_train["name"] = ticker_name
# result
df_result = dt.fread('./drive/MyDrive/millennium_project/dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [12]:
df_train = gen_row_id(df_train)
df_train = add_time_fold(df_train, N_FOLD)

In [13]:
df_train.index = df_train.row_id
df_result.index = df_result.row_id

# Evaluation

In [14]:
batch_size = 1024
learning_rate = 6e-3
epochs = 1000

list_seeds = [0, 11, 42, 777, 2045]

In [16]:
list_rmse = []
for i_seed, seed in enumerate(list_seeds):
    df_train = add_time_fold(df_train, N_FOLD, seed=seed)
    list_rmse += [[]]
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]

        X_train = df_tr[fea_cols].values
        y_train = df_tr[['value_to_predict']].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['value_to_predict']].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_seed+1}/{len(list_seeds)} | {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)


        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_{i_seed}_{i_fold}.hdf5'
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_delta=1e-5, verbose=2)
        es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=31, restore_best_weights=True, verbose=2)

        model = create_1dcnn(X_train.shape[1], 1, learning_rate)
        history = model.fit(X_train, y_train,
            epochs=epochs,
            validation_data=(X_test, y_test),
            validation_batch_size=len(y_test),
            batch_size=batch_size,
            verbose=2,
            callbacks=[rlr, es]
        )
        # model = tf.keras.models.load_model(ckp_path, custom_objects={'mspe_loss': mspe_loss})

        y_pred = model.predict(X_test, batch_size=len(y_test))
        curr_rmse = rmse_keras(y_test, y_pred)
        list_rmse[-1] += [curr_rmse]
        model.save(ckp_path)
        # generate and save preds
        df_result.loc[idx_test, f'pred_{i_seed}'] = y_pred
        clear_output()
        print(list_rmse)

[[<tf.Tensor: shape=(), dtype=float64, numpy=0.3359216795858812>, <tf.Tensor: shape=(), dtype=float64, numpy=0.3331533932595327>, <tf.Tensor: shape=(), dtype=float64, numpy=0.1290456085598979>, <tf.Tensor: shape=(), dtype=float64, numpy=0.33070284127185234>, <tf.Tensor: shape=(), dtype=float64, numpy=0.3371930732049905>, <tf.Tensor: shape=(), dtype=float64, numpy=0.11052227360671327>, <tf.Tensor: shape=(), dtype=float64, numpy=0.09187234153435685>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0885972779324477>, <tf.Tensor: shape=(), dtype=float64, numpy=0.33605726725561874>, <tf.Tensor: shape=(), dtype=float64, numpy=0.08658196393014679>], [<tf.Tensor: shape=(), dtype=float64, numpy=0.0882371471224953>, <tf.Tensor: shape=(), dtype=float64, numpy=0.3327710222406703>, <tf.Tensor: shape=(), dtype=float64, numpy=0.07821640569734854>, <tf.Tensor: shape=(), dtype=float64, numpy=0.1445896013955447>, <tf.Tensor: shape=(), dtype=float64, numpy=0.1005069112667769>, <tf.Tensor: shape=(), dtype=fl

In [19]:
df_result.to_csv(f'./drive/MyDrive/millennium_project/results/{SOL_NAME}.csv', index=False)

In [20]:
for i in range(len(list_seeds)):
    print(i, rmse(df_result['value_to_predict'], df_result[f'pred_{i}']))
print('All: ', rmse(df_result['value_to_predict'], df_result[[f'pred_{i}' for i in range(len(list_seeds))]].mean(axis=1)))

0 0.2475093231122428
1 0.22725026357894457
2 0.24868736246811837
3 0.23181703602202572
4 0.28164650636894883
All:  0.20745221373283151
